**Using template.py guide created by Roger to create a script that collects Steam reviews via public API**

Steam Guide: https://partner.steamgames.com/doc/store/getreviews

Inspired by steamreviews package https://github.com/woctezuma/download-steam-reviews

API requests limit: 10,000 per day per user https://steamcommunity.com/dev/apiterms

In [1]:
import requests
import json

# Game Steam id

def download_steam_reviews(game_id,default_cursor):
    
    appid=game_id
    base_url = "https://store.steampowered.com/appreviews/"

    # Parameters for the API request
    params = {
        "json": 1, 
        "filter": "recent", 
        "language": "english", 
        "review_type": "all", 
        "purchase_type": "all",
        "num_per_page": 100,
    }

    # Store the reviews
    reviews = dict()

    cursor = default_cursor

    # Keep track of how many requests are made
    counter = 0

    # Keep track of how many reviews are fetched
    reviews_fetched = 0
    
    print("Starting to send requests...")

    while True:
        counter += 1

        # Append the cursor to the parameters
        params["cursor"] = cursor

        try:
            response = requests.get(base_url + str(appid), params=params)

            # Check if the request was successful
            if response.status_code == 200:

                data = response.json()

                # Check if the query was successful
                if data["success"] == 1:
                    # Get user reviews
                    reviews_list = data["reviews"]

                    for review in reviews_list:

                        recommendationid = review["recommendationid"]
                        review_data = review

                        reviews[recommendationid] = review_data

                        reviews_fetched += 1

                    # Get the cursor value for the next batch of reviews
                    cursor = data["cursor"]
                    # Check if the cursor is empty or None or the same as the last cursor, meaning there are no more reviews
                    if cursor == "" or cursor == None or params["cursor"]==cursor:
                        break

                else:
                    print("Query failed")
                    break

            else:
                print("Request failed")
                break

            # Limit the max request to 150 before adding a sleep
            if counter > 9999:
                print("Limit of 10000 requests reached. Saving the collected data and the cursor.")
                with open('cursor_to_start.txt', 'w') as f:
                    f.write(cursor)
                break

        # Catch any exceptions that may occur during the request
        except Exception as e:
            print(f"An exception occurred: {e}")
            break

    # save the file
    print("Saving json with reviews ...")
    with open(str(appid)+'.json', 'a') as f:
        json.dump(reviews, f)
    print(f"Fetched {reviews_fetched} reviews for appid {appid} with {counter} requests")
    
    return


In [2]:
cursor = "*"
appid = 892970
download_steam_reviews(appid, cursor)

Starting to send requests...
Saving json with reviews ...
Fetched 244655 reviews for appid 892970 with 2449 requests
